In [1]:
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
from rasterio import features
import os
from osgeo import gdal, ogr, osr

In [2]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/RasterAnalysis.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import RasterAnalysis_bands
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import RasterAnalysis
config = RasterAnalysis_bands.Configuration()

self.trained_model_path: D:\lakemapping\5_saved_models\lakes_20231202-2055_AdaDelta_dice_loss_012345_512.h5


In [3]:
def raster2vector(raster_path, vecter_path, field_name="class", ignore_values = None):
    
    # 读取路径中的栅格数据
    raster = gdal.Open(raster_path)
    # in_band 为想要转为矢量的波段,一般需要进行转矢量的栅格都是单波段分类结果
    # 若栅格为多波段,需要提前转换为单波段
    band = raster.GetRasterBand(1)
    
    # 读取栅格的投影信息,为后面生成的矢量赋予相同的投影信息
    prj = osr.SpatialReference()
    prj.ImportFromWkt(raster.GetProjection())
    
    
    drv = ogr.GetDriverByName("ESRI Shapefile")
    # 若文件已经存在,删除
    if os.path.exists(vecter_path):
        drv.DeleteDataSource(vecter_path)
    # 创建目标文件
    polygon = drv.CreateDataSource(vecter_path)
    # 创建面图层
    poly_layer = polygon.CreateLayer(vecter_path[:-4], srs=prj, geom_type=ogr.wkbMultiPolygon)
    # 添加浮点型字段,用来存储栅格的像素值
    field = ogr.FieldDefn(field_name, ogr.OFTReal)  
    poly_layer.CreateField(field)
    
    # FPolygonize将每个像元转成一个矩形，然后将相似的像元进行合并
    # 设置矢量图层中保存像元值的字段序号为0
    gdal.FPolygonize(band, None, poly_layer, 0)
    
    # 删除ignore_value链表中的类别要素
    if ignore_values is not None:
        for feature in poly_layer:
            class_value = feature.GetField('class')
            for ignore_value in ignore_values:
                if class_value==ignore_value:
                    # 通过FID删除要素
                    poly_layer.DeleteFeature(feature.GetFID())
                    break
                
    polygon.SyncToDisk()
    polygon = None
    print('Vector File Exported Successfully!')

In [4]:
all_files = []
for root, dirs, files in os.walk(config.output_dir):
    for file in files:
        if file.endswith(config.input_image_type):
             all_files.append((os.path.join(root, file), file))
print(all_files)

for fullPath, filename in all_files:
    vecter_path =fullPath.replace('tif','shp')
    if not os.path.isfile(vecter_path) or config.overwrite_analysed_files: 
        print(vecter_path)
        field_name = "class"
#            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
        ignore_values = [0]
        raster2vector(fullPath, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values
    else:
        print('File already transformed!', fullPath)
        
print('finish')

[('H:\\result\\tif\\predition_b5_max_iew100_10000.tif', 'predition_b5_max_iew100_10000.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10001.tif', 'predition_b5_max_iew100_10001.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10002.tif', 'predition_b5_max_iew100_10002.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10003.tif', 'predition_b5_max_iew100_10003.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10004.tif', 'predition_b5_max_iew100_10004.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10005.tif', 'predition_b5_max_iew100_10005.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10006.tif', 'predition_b5_max_iew100_10006.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10007.tif', 'predition_b5_max_iew100_10007.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10008.tif', 'predition_b5_max_iew100_10008.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10009.tif', 'predition_b5_max_iew100_10009.tif'), ('H:\\result\\tif\\predition_b5_max_iew100_10010.tif', 'predition_b5_

File already transformed! H:\result\tif\predition_b5_max_iew100_10012.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10013.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10014.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10015.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10016.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10017.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10018.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10019.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10020.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10021.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10022.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10023.tif
File already transformed! H:\result\tif\predition_b5_max_iew100_10024.tif
File already transformed! H:\result\ti

Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10115.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10116.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10117.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10118.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10119.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10120.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10121.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10122.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10123.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10124.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10125.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10126.shp
Vect

Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10214.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10215.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10216.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10217.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10218.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10219.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10220.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10221.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10222.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10223.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10224.shp
Vector File Exported Successfully!
H:\result\shp\predition_b5_max_iew100_10225.shp
Vect